In [77]:
# from openptv_python.track import candsearch_in_pix_rest
from openptv_python.tracking_frame_buf import Target
from openptv_python.constants import TR_UNUSED, PT_UNUSED, MAX_CANDS, POS_INF
from openptv_python.parameters import ControlPar
from typing import List
import numpy as np

In [78]:

def candsearch_in_pix(
    next_frame: np.ndarray,
    num_targets: np.int32,
    cent_x: np.float64,
    cent_y: np.float64,
    dl: np.float64,
    dr: np.float64,
    du: np.float64,
    dd: np.float64,
    cpar: ControlPar,
) -> List[int]:
    """Search for a nearest candidate in unmatched target list."""
    
    # This is assumed in the fast binary search by y
    next_frame.sort(order = 'y')
    
    # counter = 0
    dmin = POS_INF
    p1, p2, p3, p4 = PT_UNUSED, PT_UNUSED, PT_UNUSED, PT_UNUSED 
    p = [PT_UNUSED] * MAX_CANDS
    d1, d2, d3, d4 = dmin, dmin, dmin, dmin

    xmin, xmax, ymin, ymax = cent_x - dl, cent_x + dr, cent_y - du, cent_y + dd

    xmin = max(xmin, 0.0)
    xmax = min(xmax, cpar.imx)
    ymin = max(ymin, 0.0)
    ymax = min(ymax, cpar.imy)

    if cent_x >= 0 and cent_x <= cpar.imx and cent_y >= 0 and cent_y <= cpar.imy:
        j0 = num_targets // 2
        dj = num_targets // 4
        while dj > 1:
            if next_frame[j0]['y'] < ymin:
                j0 += dj
            else:
                j0 -= dj
            dj //= 2

        j0 -= 12
        if j0 < 0:
            j0 = 0

        for j in range(j0, num_targets):
            if next_frame[j]['tnr'] != TR_UNUSED:
                if next_frame[j]['y'] > ymax:
                    break
                if xmin < next_frame[j]['x'] < xmax and ymin < next_frame[j]['y'] < ymax:
                    d = np.sqrt(
                        (cent_x - next_frame[j]['x']) ** 2
                        + (cent_y - next_frame[j]['y']) ** 2
                    )

                    if d < dmin:
                        dmin = d

                    if d < d1:
                        p4, p3, p2, p1 = p3, p2, p1, j
                        d4, d3, d2, d1 = d3, d2, d1, d
                    elif d1 < d < d2:
                        p4, p3, p2 = p3, p2, j
                        d4, d3, d2 = d3, d2, d
                    elif d2 < d < d3:
                        p4, p3 = p3, j
                        d4, d3 = d3, d
                    elif d3 < d < d4:
                        p4 = j
                        d4 = d

        p[0] = p1
        p[1] = p2
        p[2] = p3
        p[3] = p4

        # print("from inside p = ", p)

        # TODO: check why we need counter, we can use counter = len(p) - p.count(-1)
        # for j in range(4):
        #     if p[j] != -1:
        #         counter += 1

    return p

In [79]:
def candsearch_in_pix_numpy(
    next_frame: np.ndarray,
    num_targets: np.int32,
    cent_x: np.float64,
    cent_y: np.float64,
    dl: np.float64,
    dr: np.float64,
    du: np.float64,
    dd: np.float64,
    cpar: ControlPar,
) -> List[int]:

    xmin, xmax, ymin, ymax = cent_x - dl, cent_x + dr, cent_y - du, cent_y + dd

    xmin = max(xmin, 0.0)
    xmax = min(xmax, cpar.imx)
    ymin = max(ymin, 0.0)
    ymax = min(ymax, cpar.imy)
    
    ind = np.nonzero((next_frame['x'] < xmax) & (next_frame['x'] > xmin) & (next_frame['y'] < ymax) & (next_frame['y'] > ymin) & (next_frame['tnr'] != TR_UNUSED))[0]
    # print(f"{ind =}")
    order = np.argsort((next_frame[ind]['x'] - cent_x)**2 + (next_frame[ind]['y'] - cent_y)**2)
    # print(f"{order =}")
    ind = ind[order[:4].astype(int)]
    # print(f"{ind = }")
    ind = np.pad(ind, (0, 4-len(ind)), 'constant', constant_values = -999)
    
    return ind.tolist()

In [80]:

def candsearch_in_pix_rest(
    next_frame: np.ndarray,
    num_targets: int,
    cent_x: float,
    cent_y: float,
    dl: float,
    dr: float,
    du: float,
    dd: float,
    p: List[int],
    cpar: ControlPar,
) -> int:
    """Search for a nearest candidate in unmatched target list.

    Arguments:
    ---------
    next_frame - 2D numpy array of targets (pointer, x,y, n, nx,ny, sumg, track ID),
        assumed to be y sorted.
    num_targets - number of targets in the next_frame
    cent_x, cent_y - image coordinates of the position of a particle [pixel]
    dl, dr, du, dd - respectively the left, right, up, down distance
        to the search area borders from its center, [pixel]
    cpar - control_par object with attributes imx and imy.

    Returns
    -------
    int - the number of candidates found, between 0 - 1
    """
    counter = 0
    dmin = POS_INF
    xmin, xmax, ymin, ymax = cent_x - dl, cent_x + dr, cent_y - du, cent_y + dd

    xmin = max(xmin, 0.0)
    xmax = min(xmax, cpar.imx)
    ymin = max(ymin, 0.0)
    ymax = min(ymax, cpar.imy)

    if 0 <= cent_x <= cpar.imx and 0 <= cent_y <= cpar.imy:
        # binarized search for start point of candidate search
        j0, dj = num_targets // 2, num_targets // 4
        while dj > 1:
            j0 += dj if next_frame[j0]['y'] < ymin else -dj
            dj //= 2

        j0 -= 12 if j0 >= 12 else j0  # due to trunc
        for j in range(j0, num_targets):
            if next_frame[j]['tnr'] == TR_UNUSED:
                if next_frame[j]['y'] > ymax:
                    break  # finish search
                if xmin < next_frame[j]['x'] < xmax and ymin < next_frame[j]['y'] < ymax:
                    d = np.sqrt(
                        (cent_x - next_frame[j]['x']) ** 2
                        + (cent_y - next_frame[j]['y']) ** 2
                    )
                    if d < dmin:
                        dmin = d
                        p[0] = j

        if p[0] != TR_UNUSED:
            counter += 1

    return counter


In [81]:

def candsearch_in_pix_rest_numpy(
    next_frame: np.ndarray,
    num_targets: int,
    cent_x: float,
    cent_y: float,
    dl: float,
    dr: float,
    du: float,
    dd: float,
    p: List[int],
    cpar: ControlPar,
) -> int:
    """Search for a nearest candidate in unmatched target list.

    Arguments:
    ---------
    next_frame - 2D numpy array of targets (pointer, x,y, n, nx,ny, sumg, track ID),
        assumed to be y sorted.
    num_targets - number of targets in the next_frame
    cent_x, cent_y - image coordinates of the position of a particle [pixel]
    dl, dr, du, dd - respectively the left, right, up, down distance
        to the search area borders from its center, [pixel]
    cpar - control_par object with attributes imx and imy.

    Returns
    -------
    int - the number of candidates found, between 0 - 1
    """
    counter = 0
    dmin = POS_INF
    xmin, xmax, ymin, ymax = cent_x - dl, cent_x + dr, cent_y - du, cent_y + dd

    xmin = max(xmin, 0.0)
    xmax = min(xmax, cpar.imx)
    ymin = max(ymin, 0.0)
    ymax = min(ymax, cpar.imy)

    if 0 <= cent_x <= cpar.imx and 0 <= cent_y <= cpar.imy:
        # binarized search for start point of candidate search
        j0, dj = num_targets // 2, num_targets // 4
        while dj > 1:
            j0 += dj if next_frame[j0]['y'] < ymin else -dj
            dj //= 2

        j0 -= 12 if j0 >= 12 else j0  # due to trunc
        for j in range(j0, num_targets):
            print(f"{next_frame[j] = }, {j =}")
            if next_frame[j]['tnr'] == TR_UNUSED:
                if next_frame[j]['y'] > ymax:
                    break  # finish search
                if xmin < next_frame[j]['x'] < xmax and ymin < next_frame[j]['y'] < ymax:
                    d = np.sqrt(
                        (cent_x - next_frame[j]['x']) ** 2
                        + (cent_y - next_frame[j]['y']) ** 2
                    )
                    if d < dmin:
                        dmin = d
                        p[0] = j
                        print(f"{next_frame[j] = }, {j =}")

        if p[0] != TR_UNUSED:
            counter += 1

    return counter


In [82]:


test_targets = np.array([
    (0, 0.0, -0.2, 5, 1, 2, 10, -999),
    (6, 0.2, 0.2, 10, 8, 1, 20, -999),
    (3, 0.2, 0.3, 10, 3, 3, 30, -999),
    (4, 0.2, 1.0, 10, 3, 3, 40, -999),
    (1, -0.7, 1.2, 10, 3, 3, 50, -999),
    (7, 1.2, 1.3, 10, 3, 3, 60, -999),
    (5, 10.4, 2.1, 10, 3, 3, 70, -999),
], dtype=Target.dtype)
num_targets = len(test_targets)
test_targets.sort(order = 'y')
print(f"{test_targets =   }")

cent_x = 0.2
cent_y = 0.2
dl = dr = du = dd = 0.1

num_cams = 4

test_cpar = ControlPar(num_cams=num_cams)
img_format = "cam{}"
cal_format = "cal/cam{}.tif"

test_cpar.img_base_name = [""] * num_cams
test_cpar.cal_img_base_name = [""] * num_cams

for cam in range(num_cams):
    test_cpar.img_base_name[cam] = img_format.format(cam + 1)
    test_cpar.cal_img_base_name[cam] = cal_format.format(cam + 1)

test_cpar.hp_flag = 1
test_cpar.all_cam_flag = 0
test_cpar.tiff_flag = 1
test_cpar.imx = np.int32(1280)
test_cpar.imy = np.int32(1024)
test_cpar.pix_x = np.float64(0.02)
#  20 micron pixel size
test_cpar.pix_y = np.float64(0.02)
test_cpar.chfield = 0
test_cpar.mm.n1 = 1
test_cpar.mm.n2[0] = 1.49
test_cpar.mm.n3 = 1.33
test_cpar.mm.d[0] = 5

p = candsearch_in_pix(
    test_targets, num_targets, cent_x, cent_y, dl, dr, du, dd, test_cpar
)




counter = len(p) - p.count(PT_UNUSED)
print(f"p = {p}, counter = {counter}")

assert(counter == 2)

p = candsearch_in_pix_numpy(
    test_targets, num_targets, cent_x, cent_y, dl, dr, du, dd, test_cpar
)




counter = len(p) - p.count(PT_UNUSED)
print(f"p = {p}, counter = {counter}")

assert(counter == 2)




test_targets =   array([(0,  0. , -0.2,  5, 1, 2, 10, -999),
       (6,  0.2,  0.2, 10, 8, 1, 20, -999),
       (3,  0.2,  0.3, 10, 3, 3, 30, -999),
       (4,  0.2,  1. , 10, 3, 3, 40, -999),
       (1, -0.7,  1.2, 10, 3, 3, 50, -999),
       (7,  1.2,  1.3, 10, 3, 3, 60, -999),
       (5, 10.4,  2.1, 10, 3, 3, 70, -999)],
      dtype=[('pnr', '<i4'), ('x', '<f8'), ('y', '<f8'), ('n', '<i4'), ('nx', '<i4'), ('ny', '<i4'), ('sumg', '<i4'), ('tnr', '<i4')])
p = [1, 2, -999, -999], counter = 2
p = [1, 2, -999, -999], counter = 2


In [83]:

cent_x = 0.5
cent_y = 0.3
dl = dr = du = dd = 10.2


In [84]:
%%timeit
p = candsearch_in_pix(
    test_targets, num_targets, cent_x, cent_y, dl, dr, du, dd, test_cpar
)
# counter = len(p) - p.count(-1)
# print(f"p = {p}, counter = {counter}")


45.6 µs ± 3.27 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [85]:
%%timeit
p = candsearch_in_pix_numpy(
    test_targets, num_targets, cent_x, cent_y, dl, dr, du, dd, test_cpar
)
# counter = len(p) - p.count(-1)
# print(f"p = {p}, counter = {counter}")

# assert(counter == 4)


43.8 µs ± 1.3 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [86]:

test_targets = np.array([
    (0, 0.0, -0.2, 5, 1, 2, 10, 0),
    (6, 100.0, 100.0, 10, 8, 1, 20, -1),
    (3, 102.0, 102.0, 10, 3, 3, 30, -1),
    (4, 103.0, 103.0, 10, 3, 3, 40, 2),
    (1, -0.7, 1.2, 10, 3, 3, 50, 5), # should fall on imx,imy 
    (7, 1.2, 1.3, 10, 3, 3, 60, 7),
    (5, 1200, 201.1, 10, 3, 3, 70, 11),
], dtype=Target.dtype)

test_targets.sort(order="y")
print(f"Sorted {test_targets = }")

num_targets = len(test_targets)

cent_x , cent_y = 98.9, 98.9
dl = dr = du = dd = 3
p = [TR_UNUSED] * num_cams  # Initialize p

counter = candsearch_in_pix_rest(
    test_targets, num_targets, cent_x, cent_y, dl, dr, du, dd, p, test_cpar
)

print(f"p = {p}, counter = {counter}")

assert(counter == 1)
print(f"{test_targets[p[0]] = }")
assert(np.allclose(test_targets[p[0]]['x'], 100.0, rtol=1e-9))





Sorted test_targets = array([(0,  0.00e+00, -2.000e-01,  5, 1, 2, 10,  0),
       (1, -7.00e-01,  1.200e+00, 10, 3, 3, 50,  5),
       (7,  1.20e+00,  1.300e+00, 10, 3, 3, 60,  7),
       (6,  1.00e+02,  1.000e+02, 10, 8, 1, 20, -1),
       (3,  1.02e+02,  1.020e+02, 10, 3, 3, 30, -1),
       (4,  1.03e+02,  1.030e+02, 10, 3, 3, 40,  2),
       (5,  1.20e+03,  2.011e+02, 10, 3, 3, 70, 11)],
      dtype=[('pnr', '<i4'), ('x', '<f8'), ('y', '<f8'), ('n', '<i4'), ('nx', '<i4'), ('ny', '<i4'), ('sumg', '<i4'), ('tnr', '<i4')])
p = [3, -1, -1, -1], counter = 1
test_targets[p[0]] = (6, 100., 100., 10, 8, 1, 20, -1)


In [87]:

p = [TR_UNUSED] * num_cams  # Initialize p
counter = candsearch_in_pix_rest_numpy(
    test_targets, num_targets, cent_x, cent_y, dl, dr, du, dd, p, test_cpar
)

print(f"p = {p}, counter = {counter}")

# assert(counter == 1)
assert(np.allclose(test_targets[p[0]]['x'], 100.0, rtol=1e-9))


next_frame[j] = (0, 0., -0.2, 5, 1, 2, 10, 0), j =0
next_frame[j] = (1, -0.7, 1.2, 10, 3, 3, 50, 5), j =1
next_frame[j] = (7, 1.2, 1.3, 10, 3, 3, 60, 7), j =2
next_frame[j] = (6, 100., 100., 10, 8, 1, 20, -1), j =3
next_frame[j] = (6, 100., 100., 10, 8, 1, 20, -1), j =3
next_frame[j] = (3, 102., 102., 10, 3, 3, 30, -1), j =4
p = [3, -1, -1, -1], counter = 1
